In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx

import src
from src.reload import deep_reload

In [2]:
gdf_zones = gpd.read_file('Data/NGOD/Shapes/V2_Zones.shp')

gdf_zones['in_california'] = gdf_zones['V2ZONEID'].apply(lambda zid: 'CA' in zid)

gdf_zones_california = gdf_zones[gdf_zones['in_california']]

gdf_zones_california = gdf_zones_california.explode(drop = True)

In [107]:
graph = src.graph.graph_from_json('Outputs/graph.json')

In [4]:
filename = 'cities.json'

cities = src.graph.graph_from_json(filename)

In [108]:
keep = (
    list(cities.nodes) +
    [k for k, v in graph._node.items() if \
     (v.get('network', '') not in ['Tesla' 'RIVIAN_ADVENTURE', 'Non-Neworked']) and \
     (v.get('corridor', 0))]
)

graph = src.graph.subgraph(graph, keep)

In [109]:
graph.number_of_nodes(), graph.number_of_edges()

(752, 130954)

In [110]:
for n in graph._node.values():
    
    n['distance'] = 0
    n['time'] = 0
    n['price'] = 0

for s, a in graph._adj.items():
    keep = {}
    for t, l in a.items():

        l['price'] = 0
            
        if 'station' not in s:
            if l['distance'] >= 100e3:
                    keep[t] = l

        elif 'station' not in t:
            if l['distance'] >= 100e3:
                    keep[t] = l

        else:
            if 'station' in t:
                if l['distance'] >= 200e3:
                    keep[t] = l
    
    graph._adj[s] = keep

In [112]:
graph.number_of_nodes(), graph.number_of_edges(), graph.number_of_edges() / graph.number_of_nodes()

(752, 57386, 76.31117021276596)

In [113]:
deep_reload(src)

veh = src.routing.ConstrainedVehicle(
    n_cases = 30,
    risk_attitude = (0, .5),
    ess_capacity = 65 * 3.6e6,
    efficiency = 626.5,
    rate = 55e3,
)

In [114]:
deep_reload(src)

reliability = .75

nn = 0

for node in graph._node.values():

    if 'n_dcfc' in node.keys():

        if node['n_dcfc'] > 0:
            nn += 1

        station = src.routing.Station(
            veh,
            reliability = reliability,
            chargers = node['n_dcfc'],
            rate = 55e3,
        )

        node['functions'] = station.functions

In [115]:
filename = 'cities.json'

graph_cities = src.graph.graph_from_json(filename)
origin_nodes = ['Fresno']
destination_nodes = list(graph_cities.nodes)

for n in destination_nodes:
    
    graph._node[n]['min_soc'] = .5

In [116]:
deep_reload(src)

t0 = time.time()

expectations, values, paths = veh.routes(
    graph,
    origins = origin_nodes,
    destinations = destination_nodes,
    return_paths = True
)

time.time() - t0

23.046411514282227

In [90]:
pkl.dump(
    [expectations, values, paths],
    open('Outputs/Bolt_Fresno_Aggressive.pkl', 'wb')
)

In [91]:
plot_graph = src.graph.subgraph(graph, list(graph_cities.nodes))

paths_c = [v[1:-1] for k, v in paths.items() if k in graph_cities.nodes]
plot_graph_c = src.graph.subgraph(
    graph, list(np.unique(np.concatenate(paths_c)))
)

In [92]:
deep_reload(src)

path_values_avg = {}

for target, value in values.items():

    path_values_avg[target] = {}

    for key, val in value.items():

        path_values_avg[target][key] = src.routing.super_quantile(val, (0, 1))

fig, ax  = plt.subplots(figsize = (8,8))

key = 'time'

for source, node in plot_graph._node.items():

		try:
			node['plot'] = path_values_avg[source][key] / 3.6e3

		except:
			node['plot'] = np.nan

gdf_zones_california.plot(ax = ax, ec = 'gray', fc = 'none')

kwargs = {
    'show_links': False,
    'node_field': 'plot',
    'scatter': {
        's': 350,
        'ec': 'k',
        'zorder': 2,
        # 'vmin': 0,
        # 'vmax': 15,
    },
    'colorbar': {
        'label': 'Time to Node [h]',
        'ticks': range(0, 16),
    },
    'cmap': src.figures.colormap('viridis'),
}

src.figures.plot_graph(plot_graph, ax = ax, **kwargs)

# kwargs = {
# 		'node_field': None,
# 		'scatter': {
# 			's': .5,
# 			'ec': 'none',
#             'fc': 'gray',
#             'zorder': 0,
# 		},
# 		'show_links': False,
# 	}
# src.figures.plot_graph(atlas, ax = ax, **kwargs)

kwargs = {
    'scatter': {
        's': 250,
        'ec': 'k',
        'fc': 'k',
        'label': 'Charging Stations Utilized',
        'zorder': 1,
        'marker': '^',
    },
    'show_links': False,
    'axes': {
        'facecolor': 'whitesmoke',
    }
}

src.figures.plot_graph(plot_graph_c, ax = ax, **kwargs)

_ = ax.grid(ls = '--')
legend = ax.legend()
legend.legendHandles[0]._sizes = [100]

# fig.savefig('Paper/figs/R_California_Bolt_Fresno_Aggressive.png', bbox_inches = 'tight', dpi = 300)

In [93]:
{k: v['time'] / 3600 for k, v in path_values_avg.items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.818580239205238,
 'Stockton': 2.3190517545725875,
 'Sacramento': 5.263608452708013,
 'San Jose': 6.0914450276427825,
 'San Francisco': 5.91914558751905,
 'Los Angeles': 6.085232082230305,
 'Chico': 8.107291172792287,
 'I-80 to Reno (State Line)': 7.894104214381742,
 'San Diego': 10.352479370591293,
 'Redding': 10.99417197987862,
 'I-15 to Las Vegas (State Line)': 11.273697119923337,
 'I-10 to Phoenix (State Line)': 11.878729101871352,
 'Yreka': 12.44309125368885,
 'Crescent City': 17.448186230068927}

In [94]:
{k: v['time_nc'] / 3600 for k, v in path_values_avg.items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.818580239205238,
 'Stockton': 2.3190517545725875,
 'Sacramento': 3.125707124665461,
 'San Jose': 3.0724786324786337,
 'San Francisco': 3.4546262726471064,
 'Los Angeles': 3.69127664002664,
 'Chico': 4.886171247733754,
 'I-80 to Reno (State Line)': 5.236370862933366,
 'San Diego': 5.869172817922828,
 'Redding': 6.267103949291456,
 'I-15 to Las Vegas (State Line)': 6.598255517630515,
 'I-10 to Phoenix (State Line)': 7.56597149763817,
 'Yreka': 7.881089674214683,
 'Crescent City': 9.924029431216944}

In [28]:
{k: v['time'] / 3600 for k, v in path_values_avg.items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.8185802392052384,
 'Stockton': 2.3190517545725915,
 'San Jose': 3.1718551485373503,
 'Sacramento': 3.418045700078774,
 'San Francisco': 3.8699938516401367,
 'Los Angeles': 4.454643334751668,
 'Chico': 5.653357454344395,
 'I-80 to Reno (State Line)': 6.0614637638640305,
 'San Diego': 7.09528321295362,
 'Redding': 7.115529306675718,
 'I-15 to Las Vegas (State Line)': 7.949343328246323,
 'Yreka': 9.355659174872667,
 'I-10 to Phoenix (State Line)': 9.502401945764017,
 'Crescent City': 12.720386049793047}

In [29]:
{k: v['time_nc'] / 3600 for k, v in path_values_avg.items() if 'station' not in k}

{'Fresno': 0.0,
 'Bakersfield': 1.8185802392052384,
 'Stockton': 2.3190517545725915,
 'San Jose': 3.009162781662782,
 'Sacramento': 3.1022296954588615,
 'San Francisco': 3.4114510643677307,
 'Los Angeles': 3.69127664002664,
 'Chico': 4.886171247733754,
 'I-80 to Reno (State Line)': 5.187902876444547,
 'San Diego': 5.88966898841899,
 'Redding': 5.9311873889999,
 'I-15 to Las Vegas (State Line)': 6.5748180176305215,
 'Yreka': 7.564757850174525,
 'I-10 to Phoenix (State Line)': 7.559964657256322,
 'Crescent City': 10.116834997564164}